# Controller Trainer
(This part of the model does not involve any visual processing)

## Loading stuff into the kernel
Import and training data

In [1]:
# imports
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Lambda, Reshape, MaxPooling2D,LSTM, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import save_model
import numpy as np
import matplotlib.pyplot as plt
import os
import joblib
import cv2
import random
import glob

In [2]:
# loading data
data = joblib.load("images/train_data_controller.z")

In [3]:
# formatting data
# note: right now just flattens z and z' into 1 vector, the commented out code is to keep the two vectors seperate
#   I couldn't figure out how to reduce a 2x2048 input into a 1d one in a neural net
# train_data = np.array([np.array([p[0][0], p[1][0][0]]) for p in data])
train_data = np.array([[np.concatenate((p[0][0], p[1][0][0]))] for p in data])
answers = np.array([p[2] for p in data])
print(train_data[0], answers[0])

[[-3.4742382e-01  1.4214687e+00  2.9589504e-01 ...  2.3236513e-02
   9.7199380e-01  1.3059974e-03]] [22]


## Building the Controller

In [4]:
# Layers
z_len = int(len(train_data[0][0])/2)
a_len = len(answers[0])

input_to_controller = Input(shape=(1, z_len*2))

x = Dense(z_len)(input_to_controller)
x = Dropout(0.2)(x)
x = Dense(z_len/2)(x)
x = Dropout(0.2)(x)
x = Dense(z_len/4)(x)
x = Dropout(0.2)(x)
x = Dense(z_len/16)(x)
x = Dropout(0.2)(x)

output = Dense(a_len, activation='sigmoid')(x)

In [5]:
cntrl = Model(input_to_controller, output, name='controller')

In [6]:
cntrl.summary()

Model: "controller"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 4096)]         0         
                                                                 
 dense (Dense)               (None, 1, 2048)           8390656   
                                                                 
 dropout (Dropout)           (None, 1, 2048)           0         
                                                                 
 dense_1 (Dense)             (None, 1, 1024)           2098176   
                                                                 
 dropout_1 (Dropout)         (None, 1, 1024)           0         
                                                                 
 dense_2 (Dense)             (None, 1, 512)            524800    
                                                                 
 dropout_2 (Dropout)         (None, 1, 512)            0

In [7]:
cntrl.compile(optimizer='adam', loss='mean_squared_error')

In [8]:
# be sure to adjust numbers of epochs once data is better
cntrl.fit(train_data, answers, epochs=10, verbose = 1, batch_size = 32, validation_split=0.2)

Train on 603 samples, validate on 151 samples
Epoch 1/10
603/603 [==============================] - 2s 3ms/sample - loss: 441.9112 - val_loss: 441.0000
Epoch 2/10
 32/603 [>.............................] - ETA: 0s - loss: 441.0000

C:\Users\mattp\anaconda3\lib\site-packages\keras\engine\training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


603/603 [==============================] - 1s 2ms/sample - loss: 441.0000 - val_loss: 441.0000
Epoch 3/10
603/603 [==============================] - 1s 2ms/sample - loss: 441.0000 - val_loss: 441.0000
Epoch 4/10
603/603 [==============================] - 1s 2ms/sample - loss: 441.0000 - val_loss: 441.0000
Epoch 5/10
603/603 [==============================] - 1s 2ms/sample - loss: 441.0000 - val_loss: 441.0000
Epoch 6/10
603/603 [==============================] - 1s 2ms/sample - loss: 441.0000 - val_loss: 441.0000
Epoch 7/10
603/603 [==============================] - 1s 2ms/sample - loss: 441.0000 - val_loss: 441.0000
Epoch 8/10
603/603 [==============================] - 1s 2ms/sample - loss: 441.0000 - val_loss: 441.0000
Epoch 9/10
603/603 [==============================] - 1s 2ms/sample - loss: 441.0000 - val_loss: 441.0000
Epoch 10/10
603/603 [==============================] - 1s 2ms/sample - loss: 441.0000 - val_loss: 441.0000


In [9]:
cntrl.save_weights('models/cntrl.h5')